In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# 1) Load data
df = pd.read_csv("fake_reviews.csv")

# 2) Prepare text & labels
X = df["text_"].astype(str)
y = df["label"]

# Convert labels to 0/1
le = LabelEncoder()
y = le.fit_transform(y)  # OR->0, CG->1

# 3) Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4) TF-IDF vectorization
tfidf = TfidfVectorizer(max_features=5000, stop_words="english")
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

# 5) Build ANN model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # binary output
])

# Compile model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# 6) Train
model.fit(X_train_tfidf, y_train, epochs=10, batch_size=32, validation_split=0.1)

# 7) Evaluate
loss, acc = model.evaluate(X_test_tfidf, y_test)
print("Test Accuracy:", acc)


In [ ]:
import pickle

# 1️⃣ Save the ANN model
model.save("fake_review_ann_model.keras")   # recommended modern format

# 2️⃣ Save the TF-IDF vectorizer
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)

# 3️⃣ Save the Label Encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

print("Model and preprocessing files saved successfully!")


In [ ]:
from google.colab import files

files.download("fake_review_ann_model.keras")
files.download("tfidf_vectorizer.pkl")
files.download("label_encoder.pkl")
